# Project: Data Warehouse
## Setting up AWS resources

In [2]:
import pandas as pd
import boto3
import json

### Loading DWH Params from a file

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

# AWS
KEY = config.get('AWS', 'KEY')
SECRET = config.get('AWS', 'SECRET')

# IAM
IAM_ROLE_NAME = config.get('IAM_ROLE', 'ROLE_NAME')
IAM_ROLE_ARN = config.get('IAM_ROLE', 'ARN')

# DWS
DWH_CLUSTER_TYPE = config.get('DWH', 'DWH_CLUSTER_TYPE')
DWH_NUM_NODES = config.get('DWH', 'DWH_NUM_NODES')
DWH_NODE_TYPE = config.get('DWH', 'DWH_NODE_TYPE')

DWH_CLUSTER_IDENTIFIER = config.get('DWH', 'DWH_CLUSTER_IDENTIFIER')
DWH_DB = config.get('DWH', 'DWH_DB')
DWH_DB_USER = config.get('DWH', 'DWH_DB_USER')
DWH_DB_PASSWORD = config.get('DWH', 'DWH_DB_PASSWORD')
DWH_PORT = config.get('DWH', 'DWH_PORT')
DWH_HOST = config.get('DWH', 'HOST')

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, IAM_ROLE_NAME]
             })

### Creating clients and resources for IAM, EC2, S3, and Redshift


In [3]:
ec2 = boto3.resource('ec2', region_name='us-east-1', aws_access_key_id=KEY, aws_secret_access_key=SECRET)

s3 = boto3.resource('s3', region_name='us-east-1', aws_access_key_id=KEY, aws_secret_access_key=SECRET)

iam = boto3.client('iam', aws_access_key_id=KEY, aws_secret_access_key=SECRET, region_name='us-east-1')

redshift = boto3.client('redshift', region_name='us-east-1', aws_access_key_id=KEY, aws_secret_access_key=SECRET)

#### IAM ROLE
- Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [ ]:
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(Path='/', 
                              RoleName=IAM_ROLE_NAME, 
                              Description = 'Allows Redshift clusters to call AWS services in your behalf', 
                              AssumeRolePolicyDocument=json.dumps({'Statement': [{'Action':'sts:AssumeRole', 
                                                                                  'Effect': 'Allow', 
                                                                                  'Principal': {'Service':'redshift.amazonaws.com'}}], 
                                                                   'Version': '2012-10-17'}))
    
except Exception as e:
    print(e)

In [5]:
#Attaching policy
print('1.2 Attaching Policy')
iam.attach_role_policy(RoleName=IAM_ROLE_NAME, 
                       PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
                      )['ResponseMetadata']['HTTPStatusCode']

1.2 Attaching Policy


200

In [ ]:
print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

#### Redshift Cluster


In [7]:
try:
    response = redshift.create_cluster(        
        ClusterType='single-node',
        NodeType='dc2.large',
        NumberOfNodes=int(1),

        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        IamRoles=IAM_ROLE_ARN
    )
except Exception as e:
    print(e)

##### Checking cluster status

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', None)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

##### Cluster endpoint and ARN role

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
IAM_ROLE_ARN= myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", IAM_ROLE_ARN)

##### Opening an incoming TCP port to access the cluster endpoint

In [14]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName= 'default',  
        CidrIp='0.0.0.0/0',  
        IpProtocol='TCP',  
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-08c29c967d8ef4d22')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


##### Connecting the clusterConnect to the cluster

In [18]:
import redshift_connector
conn = redshift_connector.connect(
     host=DWH_HOST,
     database=DWH_DB,
     port=5439,
     user=DWH_DB_USER,
     password=DWH_DB_PASSWORD
  )
print("Conexão estabelecida com sucesso")

Conexão estabelecida com sucesso


## Cleaninup resources

In [ ]:
# run to delete the created resources
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

In [ ]:
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
# run to delete the created resources
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=IAM_ROLE_NAME)